## Решение практической задачи 1

### Задача: ранжирование потенциальных кандидатов по профессиональным обязанностям

<center>
    <img src="https://raw.githubusercontent.com/aimclub/OCEANAI/main/docs/source/user_guide/notebooks/images/Step_1_get_scores_ru.svg" width="60%"/>
</center>

<center>
    <img src="https://raw.githubusercontent.com/aimclub/OCEANAI/main/docs/source/user_guide/notebooks/images/Step_2_task_1_ru.svg" width="60%"/>
</center>

<hr>

In [1]:
import os  # Взаимодействие с файловой системой
import sys # Доступ к некоторым переменным и функциям Python

PATH_TO_SOURCE = os.path.abspath(os.path.dirname(globals()['_dh'][0]))
PATH_TO_ROOT = os.path.join(PATH_TO_SOURCE, '..', '..', '..')

sys.path.insert(0, os.path.abspath(PATH_TO_ROOT))

### `FI V2`

In [2]:
# Импорт необходимых инструментов
import os
import pandas as pd

# Импорт модуля
from oceanai.modules.lab.build import Run

# Создание экземпляра класса
_b5 = Run()

# Настройка ядра
_b5.path_to_save_ = './models' # Директория для сохранения файла
_b5.chunk_size_ = 2000000      # Размер загрузки файла из сети за 1 шаг

corpus = 'fi'

# Формирование аудиомоделей
res_load_model_hc = _b5.load_audio_model_hc()
res_load_model_nn = _b5.load_audio_model_nn()

# Загрузка весов аудиомоделей
url = _b5.weights_for_big5_['audio'][corpus]['hc']['sberdisk']
res_load_model_weights_hc = _b5.load_audio_model_weights_hc(url = url)

url = _b5.weights_for_big5_['audio'][corpus]['nn']['sberdisk']
res_load_model_weights_nn = _b5.load_audio_model_weights_nn(url = url)

# Формирование видеомоделей
res_load_model_hc = _b5.load_video_model_hc(lang='en')
res_load_model_deep_fe = _b5.load_video_model_deep_fe()
res_load_model_nn = _b5.load_video_model_nn()

# Загрузка весов видеомоделей
url = _b5.weights_for_big5_['video'][corpus]['hc']['sberdisk']
res_load_model_weights_hc = _b5.load_video_model_weights_hc(url = url)

url = _b5.weights_for_big5_['video'][corpus]['fe']['sberdisk']
res_load_model_weights_deep_fe = _b5.load_video_model_weights_deep_fe(url = url)

url = _b5.weights_for_big5_['video'][corpus]['nn']['sberdisk']
res_load_model_weights_nn = _b5.load_video_model_weights_nn(url = url)

# Загрузка словаря с экспертными признаками (текстовая модальность)
res_load_text_features = _b5.load_text_features()

# Формирование текстовых моделей 
res_setup_translation_model = _b5.setup_translation_model() # только для русского языка
res_setup_translation_model = _b5.setup_bert_encoder()
res_load_text_model_hc_fi = _b5.load_text_model_hc(corpus=corpus)
res_load_text_model_nn_fi = _b5.load_text_model_nn(corpus=corpus)

# Загрузка весов текстовых моделей
url = _b5.weights_for_big5_['text'][corpus]['hc']['sberdisk']
res_load_text_model_weights_hc_fi = _b5.load_text_model_weights_hc(url = url)

url = _b5.weights_for_big5_['text'][corpus]['nn']['sberdisk']
res_load_text_model_weights_nn_fi = _b5.load_text_model_weights_nn(url = url)

# Формирование модели для мультимодального объединения информации
res_load_avt_model_b5 = _b5.load_avt_model_b5()

# Загрузка весов модели для мультимодального объединения информации
url = _b5.weights_for_big5_['avt'][corpus]['b5']['sberdisk']
res_load_avt_model_weights_b5 = _b5.load_avt_model_weights_b5(url = url)

PATH_TO_DIR = './video_FI/'
PATH_SAVE_VIDEO = './video_FI/test/'

_b5.path_to_save_ = PATH_SAVE_VIDEO

# Загрузка 10 тестовых аудиовидеозаписей из корпуса First Impression V2
# URL: https://chalearnlap.cvc.uab.cat/dataset/24/description/
domain = 'https://download.sberdisk.ru/download/file/'
tets_name_files = [
    '429713680?token=FqHdMLSSh7zYSZt&filename=_plk5k7PBEg.003.mp4',
    '429713681?token=Hz9b4lQkrLfic33&filename=be0DQawtVkE.002.mp4',
    '429713683?token=EgUXS9Xs8xHm5gz&filename=2d6btbaNdfo.000.mp4',
    '429713684?token=1U26753kmPYdIgt&filename=300gK3CnzW0.003.mp4',
    '429713685?token=LyigAWLTzDNwKJO&filename=300gK3CnzW0.001.mp4',
    '429713686?token=EpfRbCKHyuc4HPu&filename=cLaZxEf1nE4.004.mp4',
    '429713687?token=FNTkwqBr4jOS95l&filename=g24JGYuT74A.004.mp4',
    '429713688?token=qDT95nz7hfm2Nki&filename=JZNMxa3OKHY.000.mp4',
    '429713689?token=noLguEGXDpbcKhg&filename=nvlqJbHk_Lc.003.mp4',
    '429713679?token=9L7RQ0hgdJlcek6&filename=4vdJGgZpj4k.003.mp4'
]

for curr_files in tets_name_files:
    _b5.download_file_from_url(url = domain + curr_files, out = True)

# Получение прогнозов
_b5.path_to_dataset_ = PATH_TO_DIR # Директория набора данных
_b5.ext_ = ['.mp4'] # Расширения искомых файлов

# Полный путь к файлу с верными предсказаниями для подсчета точности
url_accuracy = _b5.true_traits_[corpus]['sberdisk']

_b5.get_avt_predictions(url_accuracy = url_accuracy, lang = 'en')

<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:28:07</span><span style="color:#666">]</span> <span style="color:#666">Извлечение признаков (экспертных и нейросетевых) из текста ...</span>** 

<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:28:10</span><span style="color:#666">]</span> <span style="color:#666">Получение прогнозов и вычисление точности (мультимодальное объединение) ...</span>**<p><span style="color:#666">&nbsp;&nbsp;&nbsp;&nbsp;10 из 10 (100.0%) ... GitHub\OCEANAI\docs\source\user_guide\notebooks\video_FI\test\_plk5k7PBEg.003.mp4 ...</span></p>

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
ID,,,,,,
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.581159,0.628822,0.466609,0.622129,0.553832
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.463991,0.418851,0.41301,0.493329,0.423093
3,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.454281,0.415049,0.39189,0.485114,0.420741
4,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.588461,0.643233,0.530789,0.603038,0.593398
5,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.633433,0.533295,0.523742,0.608591,0.588456
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.636944,0.542386,0.558461,0.570975,0.558983
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.531518,0.376987,0.393309,0.4904,0.447881
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.610342,0.541418,0.563163,0.595013,0.569461
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.495809,0.458526,0.414436,0.469152,0.435461


<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:28:10</span><span style="color:#666">]</span> <span style="color:#666">Точность по отдельным персональным качествам личности человека ...</span>** 

,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Mean
Metrics,,,,,,
MAE,0.0589,0.0612,0.0864,0.0697,0.0582,0.0669
Accuracy,0.9411,0.9388,0.9136,0.9303,0.9418,0.9331


<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:28:10</span><span style="color:#666">]</span> <span style="color:#666">Средняя средних абсолютных ошибок: <span style="color:#1776D2">0.0669</span>, средняя точность: <span style="color:#1776D2">0.9331</span> ...</span>** 

<span style="color:#008001">**Лог файлы успешно сохранены ...**</span>

<span style="color:#666">**--- Время выполнения: 63.51 сек. ---**</span>

True

<hr>

Для выполнения ранжирования кандидатов необходимо знать весовые коэффициенты, определяющие приоритетность персональных качеств личности в зависимости от профессии.

Предлагаются весовые коэффициенты для 5 профессий, вычисленные на основе научных статей:

1) Sajjad H. et al. Personality and Career Choices // African Journal of Business Management. - 2012. – Vol. 6 (6) – pp. 2255-2260.
2) Alkhelil A. H. The Relationship between Personality Traits and Career Choice: A Case Study of Secondary School Students // International Journal of Academic Research in Progressive Education and Development. – 2016. – Vol. 5(2). – pp. 2226-6348.
3) De Jong N. et al. Personality Traits and Career Role Enactment: Career Role Preferences as a Mediator // Frontiers in Psychology. – 2019. – Vol. 10. – pp. 1720.

Пользователь может установить свои весовые коэффициенты; сумма весов должна быть равна 100.

In [3]:
# Загрузка датафрейма с весовыми коэффициентами
url = 'https://download.sberdisk.ru/download/file/478675798?token=fF5fNZVpthQlEV0&filename=traits_priority_for_professions.csv'
traits_priority_for_professions = pd.read_csv(url)

traits_priority_for_professions.index.name = 'ID'
traits_priority_for_professions.index += 1
traits_priority_for_professions.index = traits_priority_for_professions.index.map(str)

traits_priority_for_professions

,Profession,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
ID,,,,,,
1,Managers/executives,15,35,15,30,5
2,Entrepreneurship,30,30,5,5,30
3,Social/Non profit making professions,5,5,35,35,20
4,Public sector professions,15,50,15,15,5
5,"Scientists/researchers, and engineers",50,15,5,15,15


#### Ранжирование кандидатов на должность инженера-проектировщика

In [4]:
weights = traits_priority_for_professions.iloc[4].values[1:]
weights = list(map(int, weights))

_b5._candidate_ranking(
    weigths_openness = weights[0], 
    weigths_conscientiousness = weights[1],
    weigths_extraversion = weights[2],
    weigths_agreeableness = weights[3], 
    weigths_non_neuroticism = weights[4],
    out = False
)

_b5._save_logs(df = _b5.df_files_ranking_, name = 'engineer_candidate_ranking_fi_en', out = True)

_b5.df_files_ranking_

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Candidate score
ID,,,,,,,
5,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.633433,0.533295,0.523742,0.608591,0.588456,60.245513
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.636944,0.542386,0.558461,0.570975,0.558983,59.724659
4,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.588461,0.643233,0.530789,0.603038,0.593398,59.672006
10,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.60707,0.591893,0.520662,0.603938,0.565726,59.380182
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.610342,0.541418,0.563163,0.595013,0.569461,58.921304
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.581159,0.628822,0.466609,0.622129,0.553832,58.462735
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.531518,0.376987,0.393309,0.4904,0.447881,48.271469
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.495809,0.458526,0.414436,0.469152,0.435461,47.309692
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.463991,0.418851,0.41301,0.493329,0.423093,45.293684


#### Ранжирование кандидатов на должность руководителя

In [5]:
weights = traits_priority_for_professions.iloc[0].values[1:]
weights = list(map(int, weights))

_b5._candidate_ranking(
    weigths_openness = weights[0], 
    weigths_conscientiousness = weights[1],
    weigths_extraversion = weights[2],
    weigths_agreeableness = weights[3], 
    weigths_non_neuroticism = weights[4],
    out = False
)

_b5._save_logs(df = _b5.df_files_ranking_, name = 'executive_candidate_ranking_fi_en', out = True)

_b5.df_files_ranking_

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Candidate score
ID,,,,,,,
4,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.588461,0.643233,0.530789,0.603038,0.593398,60.360018
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.581159,0.628822,0.466609,0.622129,0.553832,59.158344
10,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.60707,0.591893,0.520662,0.603938,0.565726,58.579019
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.610342,0.541418,0.563163,0.595013,0.569461,57.249915
5,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.633433,0.533295,0.523742,0.608591,0.588456,57.222983
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.636944,0.542386,0.558461,0.570975,0.558983,56.838741
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.495809,0.458526,0.414436,0.469152,0.435461,45.95395
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.463991,0.418851,0.41301,0.493329,0.423093,44.730138
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.531518,0.376987,0.393309,0.4904,0.447881,44.018356


<hr>

Для ранжирования кандидатов по профессиональным навыкам необходимо задать по два коэффициента корреляции для каждого персонального качества личности человека и навыка, а также порога полярности качеств. Эти коэффициенты должны показывать, как измениться оценка качества человека если она больше или меньше заданного порога полярности качеств. 

В качестве примера предлагается использование коэффициентов корреляции между двумя людьми в четырьмя профессиональными навыками, представленных в статье:

1) Wehner C., de Grip A., Pfeifer H. Do recruiters select workers with different personality traits for different tasks? A discrete choice experiment // Labour Economics. - 2022. - vol. 78. - pp. 102186.

Представлены 4 профессиональных навыка: 

1) Analytical (Аналитические навыки). Умение эффективно решать новые задачи, требующие глубокого анализа. 
2) Interactive (Навыки межличностного общения). Умение убеждать и достигать компромиссов с заказчиками и коллегами.
4) Routine (Способность выполнять рутинную работу). Умение эффективно управлять рутинными задачами, соблюдая точность и внимание к деталям.
5) Non-Routine (Способность выполнять нестандартную работу). Умение реагировать и решать проблемы, не имеющие установленного порядка, проявляя адаптивность и креативные навыки в решении задач.

Пользователь может установить свои коэффициенты корреляции и ранжировать кандидатов по другим профессиональным навыкам.

#### Ранжирование кандидатов по профессиональным навыкам

In [6]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/478678231?token=0qiZwliLtHWWYMv&filename=professional_skills.csv'
df_professional_skills = pd.read_csv(url)

df_professional_skills.index.name = 'ID'
df_professional_skills.index += 1
df_professional_skills.index = df_professional_skills.index.map(str)

df_professional_skills

,Trait,Score_level,Analytical,Interactive,Routine,Non-Routine
ID,,,,,,
1,Openness,high,0.082,0.348,0.571,0.510
2,Openness,low,0.196,0.152,0.148,0.218
3,Conscientiousness,high,0.994,1.333,1.507,1.258
4,Conscientiousness,low,0.241,0.188,0.191,0.267
5,Extraversion,high,0.169,-0.060,0.258,0.017
6,Extraversion,low,0.181,0.135,0.130,0.194
7,Agreeableness,high,1.239,0.964,1.400,1.191
8,Agreeableness,low,0.226,0.180,0.189,0.259
9,Non-Neuroticism,high,0.636,0.777,0.876,0.729


In [7]:
_b5._priority_skill_calculation(
    correlation_coefficients = df_professional_skills,
    threshold = 0.5,
    out = True
)

_b5._save_logs(df = _b5.df_files_priority_skill_, name = 'skill_candidate_ranking_fi_en', out = True)

_b5.df_files_priority_skill_

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Analytical,Interactive,Routine,Non-Routine
3,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.588461,0.643233,0.530789,0.603038,0.593398,0.380379,0.414553,0.561275,0.453826
0,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.581159,0.628822,0.466609,0.622129,0.553832,0.376043,0.426703,0.539255,0.464534
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.607070,0.591893,0.520662,0.603938,0.565726,0.366839,0.398156,0.542808,0.438953
4,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.633433,0.533295,0.523742,0.608591,0.588456,0.359770,0.388761,0.533601,0.431331
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.610342,0.541418,0.563163,0.595013,0.569461,0.356558,0.383277,0.528317,0.425150
5,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.636944,0.542386,0.558461,0.570975,0.558983,0.349738,0.379180,0.522837,0.420837
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.495809,0.458526,0.414436,0.469152,0.435461,0.095773,0.074240,0.075158,0.107213
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.463991,0.418851,0.413010,0.493329,0.423093,0.093143,0.072220,0.073167,0.104315
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.454281,0.415049,0.391890,0.485114,0.420741,0.091345,0.070841,0.071797,0.102332
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.531518,0.376987,0.393309,0.490400,0.447881,0.081834,0.093685,0.118733,0.136328


### `MuPTA` (ru)

In [8]:
import os
import pandas as pd

# Импорт модуля
from oceanai.modules.lab.build import Run

# Создание экземпляра класса
_b5 = Run()

corpus = 'mupta'
lang = 'ru'

# Настройка ядра
_b5.path_to_save_ = './models' # Директория для сохранения файла
_b5.chunk_size_ = 2000000      # Размер загрузки файла из сети за 1 шаг

# Формирование аудиомоделей
res_load_model_hc = _b5.load_audio_model_hc()
res_load_model_nn = _b5.load_audio_model_nn()

# Загрузка весов аудиомоделей
url = _b5.weights_for_big5_['audio'][corpus]['hc']['sberdisk']
res_load_model_weights_hc = _b5.load_audio_model_weights_hc(url = url)

url = _b5.weights_for_big5_['audio'][corpus]['nn']['sberdisk']
res_load_model_weights_nn = _b5.load_audio_model_weights_nn(url = url)

# Формирование видеомоделей
res_load_model_hc = _b5.load_video_model_hc(lang=lang)
res_load_model_deep_fe = _b5.load_video_model_deep_fe()
res_load_model_nn = _b5.load_video_model_nn()

# Загрузка весов видеомоделей
url = _b5.weights_for_big5_['video'][corpus]['hc']['sberdisk']
res_load_model_weights_hc = _b5.load_video_model_weights_hc(url = url)

url = _b5.weights_for_big5_['video'][corpus]['fe']['sberdisk']
res_load_model_weights_deep_fe = _b5.load_video_model_weights_deep_fe(url = url)

url = _b5.weights_for_big5_['video'][corpus]['nn']['sberdisk']
res_load_model_weights_nn = _b5.load_video_model_weights_nn(url = url)

# Загрузка словаря с экспертными признаками (текстовая модальность)
res_load_text_features = _b5.load_text_features()

# Формирование текстовых моделей 
res_setup_translation_model = _b5.setup_translation_model() # только для русского языка
res_setup_translation_model = _b5.setup_bert_encoder()
res_load_text_model_hc_fi = _b5.load_text_model_hc(corpus=corpus)
res_load_text_model_nn_fi = _b5.load_text_model_nn(corpus=corpus)

# Загрузка весов текстовых моделей
url = _b5.weights_for_big5_['text'][corpus]['hc']['sberdisk']
res_load_text_model_weights_hc_fi = _b5.load_text_model_weights_hc(url = url)

url = _b5.weights_for_big5_['text'][corpus]['nn']['sberdisk']
res_load_text_model_weights_nn_fi = _b5.load_text_model_weights_nn(url = url)

# Формирование модели для мультимодального объединения информации
res_load_avt_model_b5 = _b5.load_avt_model_b5()

# Загрузка весов модели для мультимодального объединения информации
url = _b5.weights_for_big5_['avt'][corpus]['b5']['sberdisk']
res_load_avt_model_weights_b5 = _b5.load_avt_model_weights_b5(url = url)

PATH_TO_DIR = './video_MuPTA/'
PATH_SAVE_VIDEO = './video_MuPTA/test/'

_b5.path_to_save_ = PATH_SAVE_VIDEO

# Загрузка 10 тестовых аудиовидеозаписей из корпуса First Impression V2
# URL: https://chalearnlap.cvc.uab.cat/dataset/24/description/
domain = 'https://download.sberdisk.ru/download/file/'
tets_name_files = [
    '477995979?token=2cvyk7CS0mHx2MJ&filename=speaker_06_center_83.mov',
    '477995980?token=jGPtBPS69uzFU6Y&filename=speaker_01_center_83.mov',
    '477995967?token=zCaRbNB6ht5wMPq&filename=speaker_11_center_83.mov',
    '477995966?token=B1rbinDYRQKrI3T&filename=speaker_15_center_83.mov',
    '477995978?token=dEpVDtZg1EQiEQ9&filename=speaker_07_center_83.mov',
    '477995961?token=o1hVjw8G45q9L9Z&filename=speaker_19_center_83.mov',
    '477995964?token=5K220Aqf673VHPq&filename=speaker_23_center_83.mov',
    '477995965?token=v1LVD2KT1cU7Lpb&filename=speaker_24_center_83.mov',
    '477995962?token=tmaSGyyWLA6XCy9&filename=speaker_27_center_83.mov',
    '477995963?token=bTpo96qNDPcwGqb&filename=speaker_10_center_83.mov',
]

for curr_files in tets_name_files:
    _b5.download_file_from_url(url = domain + curr_files, out = True)

# Получение прогнозов
_b5.path_to_dataset_ = PATH_TO_DIR # Директория набора данных
_b5.ext_ = ['.mov'] # Расширения искомых файлов

# Полный путь к файлу с верными предсказаниями для подсчета точности
url_accuracy = _b5.true_traits_['mupta']['sberdisk']

_b5.get_avt_predictions(url_accuracy = url_accuracy, lang = lang)

<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:38:19</span><span style="color:#666">]</span> <span style="color:#666">Извлечение признаков (экспертных и нейросетевых) из текста ...</span>** 

<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:38:23</span><span style="color:#666">]</span> <span style="color:#666">Получение прогнозов и вычисление точности (мультимодальное объединение) ...</span>**<p><span style="color:#666">&nbsp;&nbsp;&nbsp;&nbsp;10 из 10 (100.0%) ... GitHub\OCEANAI\docs\source\user_guide\notebooks\video_MuPTA\test\speaker_27_center_83.mov ...</span></p>

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
ID,,,,,,
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.758137,0.693356,0.650108,0.744589,0.488671
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.681602,0.654339,0.607156,0.731282,0.417908
3,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.666104,0.656836,0.567863,0.685067,0.378102
4,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.694171,0.596195,0.571414,0.66223,0.348639
5,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.712885,0.594764,0.571709,0.716696,0.37802
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.664158,0.670411,0.60421,0.696056,0.399842
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.761213,0.652635,0.651028,0.788677,0.459676
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.692788,0.68324,0.616737,0.795205,0.447242
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.705923,0.658382,0.610645,0.697415,0.411988


<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:38:23</span><span style="color:#666">]</span> <span style="color:#666">Точность по отдельным персональным качествам личности человека ...</span>** 

,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Mean
Metrics,,,,,,
MAE,0.0673,0.0789,0.1325,0.102,0.1002,0.0962
Accuracy,0.9327,0.9211,0.8675,0.898,0.8998,0.9038


<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:38:23</span><span style="color:#666">]</span> <span style="color:#666">Средняя средних абсолютных ошибок: <span style="color:#1776D2">0.0962</span>, средняя точность: <span style="color:#1776D2">0.9038</span> ...</span>** 

<span style="color:#008001">**Лог файлы успешно сохранены ...**</span>

<span style="color:#666">**--- Время выполнения: 415.755 сек. ---**</span>

True

<hr>

Для выполнения ранжирования кандидатов необходимо знать весовые коэффициенты, определяющие приоритетность персональных качеств личности в зависимости от профессии.

Предлагаются весовые коэффициенты для 5 профессий, вычисленные на основе научных статей:

1) Sajjad H. et al. Personality and Career Choices // African Journal of Business Management. - 2012. – Vol. 6 (6) – pp. 2255-2260.
2) Alkhelil A. H. The Relationship between Personality Traits and Career Choice: A Case Study of Secondary School Students // International Journal of Academic Research in Progressive Education and Development. – 2016. – Vol. 5(2). – pp. 2226-6348.
3) De Jong N. et al. Personality Traits and Career Role Enactment: Career Role Preferences as a Mediator // Frontiers in Psychology. – 2019. – Vol. 10. – pp. 1720.

Пользователь может установить свои весовые коэффициенты; сумма весов должна быть равна 100.

In [9]:
# Загрузка датафрейма с весовыми коэффициентами
url = 'https://download.sberdisk.ru/download/file/478675798?token=fF5fNZVpthQlEV0&filename=traits_priority_for_professions.csv'
traits_priority_for_professions = pd.read_csv(url)

traits_priority_for_professions.index.name = 'ID'
traits_priority_for_professions.index += 1
traits_priority_for_professions.index = traits_priority_for_professions.index.map(str)

traits_priority_for_professions

,Profession,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
ID,,,,,,
1,Managers/executives,15,35,15,30,5
2,Entrepreneurship,30,30,5,5,30
3,Social/Non profit making professions,5,5,35,35,20
4,Public sector professions,15,50,15,15,5
5,"Scientists/researchers, and engineers",50,15,5,15,15


#### Ранжирование кандидатов на должность инженера-проектировщика

In [10]:
weights = traits_priority_for_professions.iloc[4].values[1:]
weights = list(map(int, weights))

_b5._candidate_ranking(
    weigths_openness = weights[0], 
    weigths_conscientiousness = weights[1],
    weigths_extraversion = weights[2],
    weigths_agreeableness = weights[3], 
    weigths_non_neuroticism = weights[4],
    out = False
)

_b5._save_logs(df = _b5.df_files_ranking_, name = 'engineer_candidate_ranking_mupta_ru', out = True)

_b5.df_files_ranking_

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Candidate score
ID,,,,,,,
10,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.753417,0.708372,0.654608,0.816416,0.504743,71.386866
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.758137,0.693356,0.650108,0.744589,0.488671,70.056608
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.761213,0.652635,0.651028,0.788677,0.459676,69.83061
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.692788,0.68324,0.616737,0.795205,0.447242,66.608396
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.705923,0.658382,0.610645,0.697415,0.411988,64.866121
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.681602,0.654339,0.607156,0.731282,0.417908,64.168845
5,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.712885,0.594764,0.571709,0.716696,0.37802,63.844963
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.664158,0.670411,0.60421,0.696056,0.399842,62.723563
3,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.666104,0.656836,0.567863,0.685067,0.378102,61.944574


#### Ранжирование кандидатов на должность руководителя

In [11]:
weights = traits_priority_for_professions.iloc[0].values[1:]
weights = list(map(int, weights))

_b5._candidate_ranking(
    weigths_openness = weights[0], 
    weigths_conscientiousness = weights[1],
    weigths_extraversion = weights[2],
    weigths_agreeableness = weights[3], 
    weigths_non_neuroticism = weights[4],
    out = False
)

_b5._save_logs(df = _b5.df_files_ranking_, name = 'executive_candidate_ranking_mupta_ru', out = True)

_b5.df_files_ranking_

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Candidate score
ID,,,,,,,
10,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.753417,0.708372,0.654608,0.816416,0.504743,72.929577
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.758137,0.693356,0.650108,0.744589,0.488671,70.172161
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.761213,0.652635,0.651028,0.788677,0.459676,69.984549
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.692788,0.68324,0.616737,0.795205,0.447242,69.648642
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.681602,0.654339,0.607156,0.731282,0.417908,66.261266
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.705923,0.658382,0.610645,0.697415,0.411988,65.774256
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.664158,0.670411,0.60421,0.696056,0.399842,65.370782
3,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.666104,0.656836,0.567863,0.685067,0.378102,63.941275
5,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.712885,0.594764,0.571709,0.716696,0.37802,63.476613


<hr>

Для ранжирования кандидатов по профессиональным навыкам необходимо задать по два коэффициента корреляции для каждого персонального качества личности человека и навыка, а также порога полярности качеств. Эти коэффициенты должны показывать, как измениться оценка качества человека если она больше или меньше заданного порога полярности качеств. 

В качестве примера предлагается использование коэффициентов корреляции между двумя людьми в четырьмя профессиональными навыками, представленных в статье:

1) Wehner C., de Grip A., Pfeifer H. Do recruiters select workers with different personality traits for different tasks? A discrete choice experiment // Labour Economics. - 2022. - vol. 78. - pp. 102186.

Представлены 4 профессиональных навыка: 

1) Analytical (Аналитические навыки). Умение эффективно решать новые задачи, требующие глубокого анализа. 
2) Interactive (Навыки межличностного общения). Умение убеждать и достигать компромиссов с заказчиками и коллегами.
4) Routine (Способность выполнять рутинную работу). Умение эффективно управлять рутинными задачами, соблюдая точность и внимание к деталям.
5) Non-Routine (Способность выполнять нестандартную работу). Умение реагировать и решать проблемы, не имеющие установленного порядка, проявляя адаптивность и креативные навыки в решении задач.

Пользователь может установить свои коэффициенты корреляции и ранжировать кандидатов по другим профессиональным навыкам.

#### Ранжирование кандидатов по профессиональным навыкам

In [12]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/478678231?token=0qiZwliLtHWWYMv&filename=professional_skills.csv'
df_professional_skills = pd.read_csv(url)

df_professional_skills.index.name = 'ID'
df_professional_skills.index += 1
df_professional_skills.index = df_professional_skills.index.map(str)

df_professional_skills

,Trait,Score_level,Analytical,Interactive,Routine,Non-Routine
ID,,,,,,
1,Openness,high,0.082,0.348,0.571,0.510
2,Openness,low,0.196,0.152,0.148,0.218
3,Conscientiousness,high,0.994,1.333,1.507,1.258
4,Conscientiousness,low,0.241,0.188,0.191,0.267
5,Extraversion,high,0.169,-0.060,0.258,0.017
6,Extraversion,low,0.181,0.135,0.130,0.194
7,Agreeableness,high,1.239,0.964,1.400,1.191
8,Agreeableness,low,0.226,0.180,0.189,0.259
9,Non-Neuroticism,high,0.636,0.777,0.876,0.729


In [13]:
_b5._priority_skill_calculation(
    correlation_coefficients = df_professional_skills,
    threshold = 0.5,
    out = True
)

_b5._save_logs(df = _b5.df_files_priority_skill_, name = 'skill_candidate_ranking_mupta_ru', out = True)

_b5.df_files_priority_skill_

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Analytical,Interactive,Routine,Non-Routine
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.753417,0.708372,0.654608,0.816416,0.504743,0.441817,0.469276,0.650349,0.525362
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.692788,0.683240,0.616737,0.795205,0.447242,0.383603,0.390507,0.554374,0.455371
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.761213,0.652635,0.651028,0.788677,0.459676,0.378697,0.385835,0.553319,0.453804
0,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.758137,0.693356,0.650108,0.744589,0.488671,0.376986,0.388910,0.553811,0.454611
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.681602,0.654339,0.607156,0.731282,0.417908,0.360296,0.368881,0.525020,0.430303
5,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.664158,0.670411,0.604210,0.696056,0.399842,0.353628,0.364621,0.517256,0.423307
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.705923,0.658382,0.610645,0.697415,0.411988,0.352979,0.364892,0.519516,0.425464
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.666104,0.656836,0.567863,0.685067,0.378102,0.346110,0.358763,0.507713,0.416314
4,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.712885,0.594764,0.571709,0.716696,0.378020,0.342501,0.351520,0.503399,0.413011
3,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.694171,0.596195,0.571414,0.662230,0.348639,0.327756,0.339168,0.485452,0.397089


### `MuPTA` (en)

In [14]:
import os
import pandas as pd

# Импорт модуля
from oceanai.modules.lab.build import Run

# Создание экземпляра класса
_b5 = Run()

corpus = 'fi'
lang = 'en'

# Настройка ядра
_b5.path_to_save_ = './models' # Директория для сохранения файла
_b5.chunk_size_ = 2000000      # Размер загрузки файла из сети за 1 шаг

# Формирование аудиомоделей
res_load_model_hc = _b5.load_audio_model_hc()
res_load_model_nn = _b5.load_audio_model_nn()

# Загрузка весов аудиомоделей
url = _b5.weights_for_big5_['audio'][corpus]['hc']['sberdisk']
res_load_model_weights_hc = _b5.load_audio_model_weights_hc(url = url)

url = _b5.weights_for_big5_['audio'][corpus]['nn']['sberdisk']
res_load_model_weights_nn = _b5.load_audio_model_weights_nn(url = url)

# Формирование видеомоделей
res_load_model_hc = _b5.load_video_model_hc(lang=lang)
res_load_model_deep_fe = _b5.load_video_model_deep_fe()
res_load_model_nn = _b5.load_video_model_nn()

# Загрузка весов видеомоделей
url = _b5.weights_for_big5_['video'][corpus]['hc']['sberdisk']
res_load_model_weights_hc = _b5.load_video_model_weights_hc(url = url)

url = _b5.weights_for_big5_['video'][corpus]['fe']['sberdisk']
res_load_model_weights_deep_fe = _b5.load_video_model_weights_deep_fe(url = url)

url = _b5.weights_for_big5_['video'][corpus]['nn']['sberdisk']
res_load_model_weights_nn = _b5.load_video_model_weights_nn(url = url)

# Загрузка словаря с экспертными признаками (текстовая модальность)
res_load_text_features = _b5.load_text_features()

# Формирование текстовых моделей 
res_setup_translation_model = _b5.setup_translation_model() # только для русского языка
res_setup_translation_model = _b5.setup_bert_encoder()
res_load_text_model_hc_fi = _b5.load_text_model_hc(corpus=corpus)
res_load_text_model_nn_fi = _b5.load_text_model_nn(corpus=corpus)

# Загрузка весов текстовых моделей
url = _b5.weights_for_big5_['text'][corpus]['hc']['sberdisk']
res_load_text_model_weights_hc_fi = _b5.load_text_model_weights_hc(url = url)

url = _b5.weights_for_big5_['text'][corpus]['nn']['sberdisk']
res_load_text_model_weights_nn_fi = _b5.load_text_model_weights_nn(url = url)

# Формирование модели для мультимодального объединения информации
res_load_avt_model_b5 = _b5.load_avt_model_b5()

# Загрузка весов модели для мультимодального объединения информации
url = _b5.weights_for_big5_['avt'][corpus]['b5']['sberdisk']
res_load_avt_model_weights_b5 = _b5.load_avt_model_weights_b5(url = url)

PATH_TO_DIR = './video_MuPTA/'
PATH_SAVE_VIDEO = './video_MuPTA/test/'

_b5.path_to_save_ = PATH_SAVE_VIDEO

# Загрузка 10 тестовых аудиовидеозаписей из корпуса First Impression V2
# URL: https://chalearnlap.cvc.uab.cat/dataset/24/description/
domain = 'https://download.sberdisk.ru/download/file/'
tets_name_files = [
    '477995979?token=2cvyk7CS0mHx2MJ&filename=speaker_06_center_83.mov',
    '477995980?token=jGPtBPS69uzFU6Y&filename=speaker_01_center_83.mov',
    '477995967?token=zCaRbNB6ht5wMPq&filename=speaker_11_center_83.mov',
    '477995966?token=B1rbinDYRQKrI3T&filename=speaker_15_center_83.mov',
    '477995978?token=dEpVDtZg1EQiEQ9&filename=speaker_07_center_83.mov',
    '477995961?token=o1hVjw8G45q9L9Z&filename=speaker_19_center_83.mov',
    '477995964?token=5K220Aqf673VHPq&filename=speaker_23_center_83.mov',
    '477995965?token=v1LVD2KT1cU7Lpb&filename=speaker_24_center_83.mov',
    '477995962?token=tmaSGyyWLA6XCy9&filename=speaker_27_center_83.mov',
    '477995963?token=bTpo96qNDPcwGqb&filename=speaker_10_center_83.mov',
]

for curr_files in tets_name_files:
    _b5.download_file_from_url(url = domain + curr_files, out = True)

# Получение прогнозов
_b5.path_to_dataset_ = PATH_TO_DIR # Директория набора данных
_b5.ext_ = ['.mov'] # Расширения искомых файлов

# Полный путь к файлу с верными предсказаниями для подсчета точности
url_accuracy = _b5.true_traits_['mupta']['sberdisk']

_b5.get_avt_predictions(url_accuracy = url_accuracy, lang = lang)

<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:47:11</span><span style="color:#666">]</span> <span style="color:#666">Извлечение признаков (экспертных и нейросетевых) из текста ...</span>** 

<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:47:14</span><span style="color:#666">]</span> <span style="color:#666">Получение прогнозов и вычисление точности (мультимодальное объединение) ...</span>**<p><span style="color:#666">&nbsp;&nbsp;&nbsp;&nbsp;10 из 10 (100.0%) ... GitHub\OCEANAI\docs\source\user_guide\notebooks\video_MuPTA\test\speaker_27_center_83.mov ...</span></p>

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
ID,,,,,,
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.564985,0.539052,0.440615,0.59251,0.488763
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.650774,0.663849,0.607308,0.643847,0.620627
3,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.435976,0.486683,0.313828,0.415446,0.396618
4,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.498542,0.511243,0.412592,0.468947,0.44399
5,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.394776,0.341608,0.327082,0.427304,0.354936
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.566107,0.543811,0.492766,0.587411,0.499433
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.506271,0.438215,0.430894,0.456177,0.44075
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.486463,0.521755,0.309894,0.432291,0.433601
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.417404,0.473339,0.320714,0.445086,0.414649


<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:47:14</span><span style="color:#666">]</span> <span style="color:#666">Точность по отдельным персональным качествам личности человека ...</span>** 

,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Mean
Metrics,,,,,,
MAE,0.1727,0.1672,0.1661,0.2579,0.107,0.1742
Accuracy,0.8273,0.8328,0.8339,0.7421,0.893,0.8258


<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:47:14</span><span style="color:#666">]</span> <span style="color:#666">Средняя средних абсолютных ошибок: <span style="color:#1776D2">0.1742</span>, средняя точность: <span style="color:#1776D2">0.8258</span> ...</span>** 

<span style="color:#008001">**Лог файлы успешно сохранены ...**</span>

<span style="color:#666">**--- Время выполнения: 374.358 сек. ---**</span>

True

<hr>

Для выполнения ранжирования кандидатов необходимо знать весовые коэффициенты, определяющие приоритетность персональных качеств личности в зависимости от профессии.

Предлагаются весовые коэффициенты для 5 профессий, вычисленные на основе научных статей:

1) Sajjad H. et al. Personality and Career Choices // African Journal of Business Management. - 2012. – Vol. 6 (6) – pp. 2255-2260.
2) Alkhelil A. H. The Relationship between Personality Traits and Career Choice: A Case Study of Secondary School Students // International Journal of Academic Research in Progressive Education and Development. – 2016. – Vol. 5(2). – pp. 2226-6348.
3) De Jong N. et al. Personality Traits and Career Role Enactment: Career Role Preferences as a Mediator // Frontiers in Psychology. – 2019. – Vol. 10. – pp. 1720.

Пользователь может установить свои весовые коэффициенты; сумма весов должна быть равна 100.

In [15]:
# Загрузка датафрейма с весовыми коэффициентами
url = 'https://download.sberdisk.ru/download/file/478675798?token=fF5fNZVpthQlEV0&filename=traits_priority_for_professions.csv'
traits_priority_for_professions = pd.read_csv(url)

traits_priority_for_professions.index.name = 'ID'
traits_priority_for_professions.index += 1
traits_priority_for_professions.index = traits_priority_for_professions.index.map(str)

traits_priority_for_professions

,Profession,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
ID,,,,,,
1,Managers/executives,15,35,15,30,5
2,Entrepreneurship,30,30,5,5,30
3,Social/Non profit making professions,5,5,35,35,20
4,Public sector professions,15,50,15,15,5
5,"Scientists/researchers, and engineers",50,15,5,15,15


#### Ранжирование кандидатов на должность инженера-проектировщика

In [16]:
weights = traits_priority_for_professions.iloc[4].values[1:]
weights = list(map(int, weights))

_b5._candidate_ranking(
    weigths_openness = weights[0], 
    weigths_conscientiousness = weights[1],
    weigths_extraversion = weights[2],
    weigths_agreeableness = weights[3], 
    weigths_non_neuroticism = weights[4],
    out = False
)

_b5._save_logs(df = _b5.df_files_ranking_, name = 'engineer_candidate_ranking_mupta_en', out = True)

_b5.df_files_ranking_

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Candidate score
ID,,,,,,,
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.650774,0.663849,0.607308,0.643847,0.620627,64.500115
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.566107,0.543811,0.492766,0.587411,0.499433,55.228999
10,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.526112,0.661107,0.443167,0.558965,0.554224,55.135903
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.564985,0.539052,0.440615,0.59251,0.488763,54.757229
4,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.498542,0.511243,0.412592,0.468947,0.44399,48.352751
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.506271,0.438215,0.430894,0.456177,0.44075,47.495148
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.486463,0.521755,0.309894,0.432291,0.433601,46.68731
3,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.435976,0.486683,0.313828,0.415446,0.396618,42.849135
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.417404,0.473339,0.320714,0.445086,0.414649,42.469887


#### Ранжирование кандидатов на должность руководителя

In [17]:
weights = traits_priority_for_professions.iloc[0].values[1:]
weights = list(map(int, weights))

_b5._candidate_ranking(
    weigths_openness = weights[0], 
    weigths_conscientiousness = weights[1],
    weigths_extraversion = weights[2],
    weigths_agreeableness = weights[3], 
    weigths_non_neuroticism = weights[4],
    out = False
)

_b5._save_logs(df = _b5.df_files_ranking_, name = 'executive_candidate_ranking_mupta_en', out = True)

_b5.df_files_ranking_

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Candidate score
ID,,,,,,,
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.650774,0.663849,0.607308,0.643847,0.620627,64.5245
10,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.526112,0.661107,0.443167,0.558965,0.554224,57.218016
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.566107,0.543811,0.492766,0.587411,0.499433,55.035993
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.564985,0.539052,0.440615,0.59251,0.488763,54.169956
4,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.498542,0.511243,0.412592,0.468947,0.44399,47.84888
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.486463,0.521755,0.309894,0.432291,0.433601,45.343512
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.506271,0.438215,0.430894,0.456177,0.44075,45.28406
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.417404,0.473339,0.320714,0.445086,0.414649,43.064455
3,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.435976,0.486683,0.313828,0.415446,0.396618,42.72743


<hr>

Для ранжирования кандидатов по профессиональным навыкам необходимо задать по два коэффициента корреляции для каждого персонального качества личности человека и навыка, а также порога полярности качеств. Эти коэффициенты должны показывать, как измениться оценка качества человека если она больше или меньше заданного порога полярности качеств. 

В качестве примера предлагается использование коэффициентов корреляции между двумя людьми в четырьмя профессиональными навыками, представленных в статье:

1) Wehner C., de Grip A., Pfeifer H. Do recruiters select workers with different personality traits for different tasks? A discrete choice experiment // Labour Economics. - 2022. - vol. 78. - pp. 102186.

Представлены 4 профессиональных навыка: 

1) Analytical (Аналитические навыки). Умение эффективно решать новые задачи, требующие глубокого анализа. 
2) Interactive (Навыки межличностного общения). Умение убеждать и достигать компромиссов с заказчиками и коллегами.
4) Routine (Способность выполнять рутинную работу). Умение эффективно управлять рутинными задачами, соблюдая точность и внимание к деталям.
5) Non-Routine (Способность выполнять нестандартную работу). Умение реагировать и решать проблемы, не имеющие установленного порядка, проявляя адаптивность и креативные навыки в решении задач.

Пользователь может установить свои коэффициенты корреляции и ранжировать кандидатов по другим профессиональным навыкам.

#### Ранжирование кандидатов по профессиональным навыкам

In [18]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/478678231?token=0qiZwliLtHWWYMv&filename=professional_skills.csv'
df_professional_skills = pd.read_csv(url)

df_professional_skills.index.name = 'ID'
df_professional_skills.index += 1
df_professional_skills.index = df_professional_skills.index.map(str)

df_professional_skills

,Trait,Score_level,Analytical,Interactive,Routine,Non-Routine
ID,,,,,,
1,Openness,high,0.082,0.348,0.571,0.510
2,Openness,low,0.196,0.152,0.148,0.218
3,Conscientiousness,high,0.994,1.333,1.507,1.258
4,Conscientiousness,low,0.241,0.188,0.191,0.267
5,Extraversion,high,0.169,-0.060,0.258,0.017
6,Extraversion,low,0.181,0.135,0.130,0.194
7,Agreeableness,high,1.239,0.964,1.400,1.191
8,Agreeableness,low,0.226,0.180,0.189,0.259
9,Non-Neuroticism,high,0.636,0.777,0.876,0.729


In [19]:
_b5._priority_skill_calculation(
    correlation_coefficients = df_professional_skills,
    threshold = 0.5,
    out = True
)

_b5._save_logs(df = _b5.df_files_priority_skill_, name = 'skill_candidate_ranking_mupta_en', out = True)

_b5.df_files_priority_skill_

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Analytical,Interactive,Routine,Non-Routine
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.650774,0.663849,0.607308,0.643847,0.620627,0.401662,0.435568,0.594751,0.479320
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.526112,0.661107,0.443167,0.558965,0.554224,0.365108,0.418729,0.524472,0.451144
5,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.566107,0.543811,0.492766,0.587411,0.499433,0.301469,0.326821,0.422422,0.377380
0,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.564985,0.539052,0.440615,0.592510,0.488763,0.299438,0.324710,0.420577,0.374751
3,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.498542,0.511243,0.412592,0.468947,0.443990,0.175691,0.193594,0.212040,0.211799
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.486463,0.521755,0.309894,0.432291,0.433601,0.171503,0.191607,0.210450,0.207539
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.417404,0.473339,0.320714,0.445086,0.414649,0.088071,0.068355,0.069366,0.098712
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.435976,0.486683,0.313828,0.415446,0.396618,0.087107,0.067595,0.068527,0.097573
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.506271,0.438215,0.430894,0.456177,0.440750,0.083889,0.093786,0.117636,0.136369
4,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.394776,0.341608,0.327082,0.427304,0.354936,0.077790,0.060347,0.061175,0.087174
